# Robotic Arm



A [Robotic arm](https://en.wikipedia.org/wiki/Robotic_arm) is a type of mechanical arm, usually programmable, with similar characteristics of a human arm. The purpose of the robot is to replace the human in difficult and harmful tasks e.g., Industrial production, processing and manufacturing roles. Any task in which extremely precise, fast and repeatable movements are required as well as to automate the process of doing the daily task that humans typically do.

The [arm](https://www.intel.com/content/www/us/en/robotics/robotic-arm.html#:~:text=Most%20industrial%20robotic%20arms%20use,power%20systems%2C%20and%20software%20components) 
contains:
- __Motors and gears__ that allow arm to rotate/move in different directions.
- __Joints__ that consists of several joints.
- __Links__ are the rigid segments that connect the joints.
- __End-effector__ is the device that is attached to the arm's last link e.g., grippers to pick up items.
- __Sensors__, such as cameras, force sensors, or proximity sensors, which allow the arm to sense and respond to its environment.
- __Control system__ is the brain of the robotic arm, which directs the movement and operation of the arm. It is commonly run in a computer, software, and control electronics.

# Control robot arm through a mobile application
In this code [here](https://github.com/Julieanna97/Robot-Arm-Research/blob/master/Mobile%20App/Mobile_app.cpp), the __read_signals_from_moile_app()__ function reads the signals from the mobile application to the robot arm in binary string, which is converted using __stoi__ with a base of 2. 

It sends one uint16 (__2__ bytes) at a time, encoded like this:
- bit __1__ - __SIGNAL_UP__
- bit __2__ - __SIGNAL_DOWN__
- bit __4__ - __SIGNAL_LEFT__
- bit __8__ - __SIGNAL_RIGHT__

I'm coding this using the bitwise operation (boolean algebra) to get signals in the right place. For example, the robot arm moves to the right by shifting 3 bits to the left. To switch the robot arm's position, I use bitshift and bitwise right shift. See the code below.

In [ ]:
// Define the signals to control the robot arm
const int SIGNAL_UP    = 1 << 0; // 00000001b
const int SIGNAL_DOWN  = 1 << 1; // 00000010b
const int SIGNAL_LEFT  = 1 << 2; // 00000100b
const int SIGNAL_RIGHT = 1 << 3; // 00001000b

In [ ]:
// Function to read signals from mobile app (replace with actual code)
int read_signals_from_mobile_app() {
    // For example, if the signals are sent as a binary string
    string signals_str = "1000"; // Up and right signals are active
    int signals = stoi(signals_str, nullptr, 2);
    return signals;
}

The __control_robot_arm()__ function takes an __integer argument__ that contains the signals and uses __bitwise__ operations to determine which signals are __active__. The function then uses __conditional statements__ (logical operator) to control the robot arm based on the active signals.

In [ ]:
// Function to control the robot arm based on signals
void control_robot_arm(int signals) {
    // Determine which signals are active using bitwise operations
    bool up    = (signals & SIGNAL_UP)    != 0;
    bool down  = (signals & SIGNAL_DOWN)  != 0;
    bool left  = (signals & SIGNAL_LEFT)  != 0;
    bool right = (signals & SIGNAL_RIGHT) != 0;

    // Use the signals to control the robot arm (replace with actual code)
    // For example, if there is a function called "control_robot_arm(string command)"
    // You could do something like this:
    if (up && !down && !left && !right) {
        cout << "Moving robot arm up" << endl;
    } else if (!up && down && !left && !right) {
        cout << "Moving robot arm down" << endl;
    } else if (!up && !down && left && !right) {
        cout << "Moving robot arm left" << endl;
    } else if (!up && !down && !left && right) {
        cout << "Moving robot arm right" << endl;
    } else {
        cout << "Stopping robot arm" << endl;
    }
}

In the __main()__ function, the program calls __read_signals_from_mobile_app()__ to read the signals from the mobile application and then calls __control_robot_arm()__ with the signals as an argument to control the robot arm. 

In [ ]:
int main() {
    // Read the signals from the mobile application
    int signals = read_signals_from_mobile_app();
    cout << "Signals received: " << bitset<4>(signals) << endl;

    // Control the robot arm based on the signals
    control_robot_arm(signals);

    return 0;
}

__Output:__

Signals received: 1000

Moving robot arm right

__Note:__ This is just a simple example and you would need to replace the placeholder code with actual code to read signals from a mobile application using a communication protocol such as __Bluetooth__ or __Wi-Fi__. _Additionally_, you may need to adapt this code to the specific requirements fo your robot arm and mobile application.

# Navigation using __LIDAR__
Robot arms use a _variety_ of sensors to be able to navigate different objects. By measuring distance to various objects to detect obstacles such as walls, furnitures etc. It provides real-time monitoring and feedback to the robot arm's control system, allowing it to adjust its movements and behavior based on the environment. It also increases the autonomy of the robot arm by allowing it to navigate and operate in complex environments without human intervention. This can be particularly useful in warehousing, logistics, or manufacturing, where robots need to operate in dynamic and unpredictable environments.

![](https://eu-images.contentstack.com/v3/assets/blt31d6b0704ba96e9d/blt4c46a271d01eb143/63abebe3d25aa95489bdd438/VelodyneLidar_DeliveryRobot_FoV.jpg)

- In order to calculate the distance to the object we can use laser pulses to bounce to the object and then back to the sensor. We can use the equation to calculate the distance based on speed and time. However, since the electromagnetic waves go back and forth, we must divide by __two__ to get the distance to the object. Here is the formula:

$$ distance = \frac{speed-of-light \times time-of-flight} {2}  $$

__Where:__
- __distance__ is the distance between the lidar sensor and the object
- the speed of __electromagnetic radiation__ is the same _as_ the _constant_ __speed of light__ in a vacuum: $$ 299,792,458m/s$$
- [__time-of-flight__](https://www.analog.com/en/applications/technology/3d-time-of-flight.html#:~:text=3D%20time%20of%20flight%20(ToF)%20is%20a%20type%20of%20scanner,from%20a%20scene%20of%20interest) is the time it _takes_ for the __laser pulse__ to travel from the lidar sensor to the object and back to the sensor. It uses a __high power optical pulses__ in durations of __nanoseconds__: 10^-9 = $$0.000000001$$

![time-of-flight](https://www.analog.com/-/media/analog/en/technologies/3d-time-of-flight-solution/tof-example2.svg?la=en&imgver=1)

We can write a function that calculates the distance to an object from a measured time-of-flight:

In [ ]:
#include <iostream>
#include <cmath>

using namespace std;

int main()
{
    const double speed_of_light = 299792458.0;  // Speed of light in m/s
    double time_of_flight;                     // Time it takes for the laser pulse to travel to the object and back in seconds
    double distance;                           // Distance between the LIDAR sensor and the object in meters
    
    // Read in the measured time-of-flight in seconds from the LIDAR sensor
    cout << "Enter time-of-flight in seconds: ";
    cin >> time_of_flight;
    
    // Calculate the distance to the object using the time-of-flight and the speed of light
    distance = (speed_of_light * time_of_flight) / 2.0;
    
    // Output the calculated distance to the object in meters
    cout << "Distance to object is: " << distance << " meters" << endl;
    
    return 0;
}

Example:

30 ns (nanoseconds) = 3.0 x 10^-9 s

__Input:__ 0.000000003

__Output:__  Distance to object is: 0.449689 meters

# Conversion from A/D

![](https://ars.els-cdn.com/content/image/3-s2.0-B9780128150719000026-f02-02-9780128150719.jpg)

The ADC (Analog-to-Digital) is used to convert analog sensor readings, such as precise position, velocity or force, into electrical/digital signals that can be processed by the robot's control system. The DAC (Digital-to-Analog), is used to convert digital control signals from the robot's control system into analog control signals that can be sent to the robot's actuators with high precision.

Many robotic arms use sensors i.e., encoders or force sensors, to provide feedback about the robot's position and force. ADC is used to covert these analog sensor readings into digital signals that can be processed by the robot's control system. The control system can then use the digital signals to adjust the robot's movements and maintain precise control.

![Digital Signal](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Digital-signal-noise.svg/1200px-Digital-signal-noise.svg.png)

![](https://cdn1.byjus.com/wp-content/uploads/2022/05/Difference-Between-Analog-And-Digital-Signal.png)

A __digital signal__ is a signal that uses binary to represent information. The information is represented as a series of __binary__ digits or __"bits"__ which is made of __1s__ and __0s__.

To convert to digital, I will need to use the formula for ADC:

# Problem 1: Accuracy

The accuracy of a robot arm in terms of position and orientation refers to how precisely the arm can move to a specific location and how accurately it can orient its end-effector in a particular direction. This accuracy is typically measured repeatedly along with its positional accuracy.

The accuracy issue is based on the object's displacement from the target area after a series of repeated movements by the arm. It could be error in accuracy along the X or Y-axis due to sensor errors, design, mechanical wear and tear, control algorithms etc.

We can measure the error by:
- [Euclidean system](https://www.kth.se/polopoly_fs/1.1078075.1622113235!/gr22rapport2021-02.pdf) by comparing the __position__ and __orientation__ of the end-effector to the target position and orientation to see the _displacement_ in both __x, y & z__ directions.

![Model](https://i.pcmag.com/imagery/encyclopedia-terms/x-y-z-matrix-xymatrix.fit_lim.size_1050x.gif)

measured along:

x = eastwest axis

y = northsouth axis

z = height/elevation

Calculation for Eculidean distance between each target point and the measure point:
$$ AP_p = {\sqrt[2]{(\bar{x1}-x2_c)^2+(\bar{y1}-y2_c)^2+(\bar{z1}-z2_c)^2}} $$

Code [here](https://github.com/Julieanna97/Robot-Arm-Research/blob/master/Accuracy%20measurement/Accuracy_Euclidean.cpp)

Example of table _positions_ of x, y & z:

<!-- Target Position Table -->
| $X-Target$ | $Y-Target$ | $Z-Target$ |
| -----| ------| ------|
| $X1 = 10.0$ | $Y1 = 6.0$ | $Z1 = 4.0$ |
-
<!-- Starting Position Table -->
| $X-Measured$ | $Y-Measured$ | $Z-Measured$ |
| -----| ------| ------|
| $X2 = 9.8$ | $Y2 = 6.1$ | $Z2 = 4.2$ |

Where $\bar{x1}-x2_c, \bar{y1}-y2_c, \bar{z1}-z2_c$ is the error along the x, y and z-axis.

$ Calculation: $
$$ AP_p = {\sqrt[2]{(\bar{10.0}-9.8_c)^2+(\bar{6.0}-6.1_c)^2+(\bar{4.0}-4.2_c)^2}} $$

We get the following value:

$$ 0.04 + 0.01 + 0.04 = 0,09$$

Square root of 2 = $${\sqrt[2]{0,09}} => 0.3$$

To calculate the average/mean distance as a measure of the accuracy of the robotic arm:
$$ m = \frac{sum}{distance}$$

Example:
sum = 4.5
$$ m = \frac{4.5}{0.3} = 15 $$
__X & Y__ can be measured in __millimeters__, __centimeters__ or __inches__, depending on the application and the precision __required__. Robotic arms are required to be __highly__ precise, with __accuracy__ levels measured in _millimeter_ or even _microns_.

Lastly, we calculate the accuracy error as the difference between the average distance and the ideal distance of e.g., __7.0__ using absolute.
$$ abs(15 - 7.0) $$
So the __accuracy error__ is: __8__